In [1]:
__author__ = 'aqeel'
import numpy as np
import os
from keras.models import load_model
from ALutils import RANK, Calculate_Score
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import math
np.random.seed(1377)

Using TensorFlow backend.


# Area To Check All Models Results on All data

In [2]:
FUll_Results = pd.read_csv('Results.csv')

In [3]:
FUll_Results[FUll_Results['Perfect Score']==FUll_Results[FUll_Results['Data Type']=='TEST']['Perfect Score'].max()]

,Model,Perfect Score,Predicted Score,Data Type
2351,Model_LSTM_07-0.06.h5,18.96678,33.16822,TEST


In [6]:
FUll_Results[FUll_Results['Data Type']=='TEST'].sort_values(['Perfect Score'],ascending=False)

,Model,Perfect Score,Predicted Score,Data Type
2351,Model_LSTM_07-0.06.h5,18.966780,33.16822,TEST
7405,Model_LSTM3_12-0.06.h5,18.880700,33.16822,TEST
6266,Model_LSTM8_24-0.06.h5,18.827540,33.16822,TEST
556,Model_LSTM_10-0.06.h5,18.809580,33.16822,TEST
3447,Model_LSTM6_82-0.05.h5,18.804720,33.16822,TEST
4003,Model_LSTM6_32-0.06.h5,18.764820,33.16822,TEST
12600,Model_LSTM6_57-0.06.h5,18.747460,33.16822,TEST
5532,Model_LSTM6_28-0.06.h5,18.708000,33.16822,TEST
2201,Model_LSTM_04-0.06.h5,18.674880,33.16822,TEST
8903,Model_LSTM6_52-0.06.h5,18.650770,33.16822,TEST


In [7]:
data=[]
#x = ['Model_LSTM_07-0.06.h5','Model_LSTM3_28-0.06.h5','Model_LSTM6_82-0.05.h5']
#for i in x:#FUll_Results['Model'].unique():
for i in FUll_Results['Model'].unique():
    t = FUll_Results[FUll_Results['Model']==i]['Perfect Score']
    tstresult = FUll_Results[(FUll_Results['Model']==i)&(FUll_Results['Data Type']=='TEST')]['Perfect Score'].as_matrix()
    tup = (i[6:],t.mean(),t.median(),t.min(),t.max(),tstresult,t.std())
    data.append(tup)
    #print('M:{},AR:{},MedR:{},MiR:{},MXR:{}'.format(tup[0],tup[1],tup[2],tup[3],tup[4]))

In [11]:
for i in data:
    print(i)

('NN:2_67-0.12.h5', 15.157017338129496, 16.11437, 3.6873769999999997, 20.507739999999998, array([ 17.07207]), 3.7226905113749122)
('LSTM2_10-0.06.h5', 15.959024338582676, 17.23467, 4.6822809999999997, 20.291409999999999, array([ 17.34103]), 3.4567938905945099)
('LSTM_00-0.08.h5', 14.371544751633985, 15.232579999999999, 3.198283, 19.9633, array([ 15.61657]), 3.8220859128225477)
('LSTM2_20-0.06.h5', 16.238302116788322, 17.47609, 4.4110050000000003, 20.635670000000001, array([ 17.62523]), 3.7723350582786956)
('LSTM_10-0.06.h5', 16.358902851612903, 17.6766, 3.7875480000000001, 20.549970000000002, array([ 18.80958]), 3.8795161240510523)
('LSTM_01-0.07.h5', 14.337498489655172, 15.431479999999999, 3.1879060000000004, 20.149260000000002, array([ 15.65636]), 3.7785960538274868)
('LSTM2_05-0.07.h5', 15.277192402597406, 16.494735, 4.0562800000000001, 20.19924, array([ 17.17989]), 3.6892482268990623)
('LSTM2_15-0.06.h5', 15.938443809090909, 17.21699, 4.5358859999999996, 20.598400000000002, array([

In [8]:
print('Model With Maximum Average')
print(sorted(data, key=lambda x: x[1],reverse=True)[0])
print('Model With Maximum Median')
print(sorted(data, key=lambda x: x[2],reverse=True)[0])
print('Model With Maximum minimum')
print(sorted(data, key=lambda x: x[3],reverse=True)[0])
print('Model With Maximum max')
print(sorted(data, key=lambda x: x[4],reverse=True)[0])
print(sorted(data, key=lambda x: x[5],reverse=True)[0])

Model With Maximum Average
('LSTM3_28-0.06.h5', 17.285850237179492, 18.46808, 5.4808510000000004, 21.130379999999999, array([ 17.43621]), 3.37599342055413)
Model With Maximum Median
('LSTM3_28-0.06.h5', 17.285850237179492, 18.46808, 5.4808510000000004, 21.130379999999999, array([ 17.43621]), 3.37599342055413)
Model With Maximum minimum
('LSTM3_28-0.06.h5', 17.285850237179492, 18.46808, 5.4808510000000004, 21.130379999999999, array([ 17.43621]), 3.37599342055413)
Model With Maximum max
('LSTM9_24-0.06.h5', 16.329193992857142, 17.483655, 5.4391819999999997, 21.172460000000001, array([ 15.97445]), 3.690459091675351)
('LSTM2_00-0.08.h5', 13.238535820512819, 13.95234, 3.8038220000000003, 19.910610000000002, array([], dtype=float64), 3.5693030012056219)


In [24]:
for i in data:
    print(i)

('LSTM_07-0.06.h5', 15.745453294117647, 16.81371, 3.7798660000000002, 20.742439999999998, array([ 18.96678]), 3.8202085745816303)
('LSTM3_28-0.06.h5', 17.285850237179492, 18.46808, 5.4808510000000004, 21.130379999999999, array([ 17.43621]), 3.37599342055413)
('LSTM6_82-0.05.h5', 16.583094486111108, 17.666040000000002, 4.5193349999999999, 20.38325, array([ 18.80472]), 3.6747361014508768)


In [30]:
sorted([(1,2),(1,4),(5,7),(1,-10)],key=lambda x:x[1])

[(1, -10), (1, 2), (1, 4), (5, 7)]

# Here Generating the Results and Storing them.
# Prepare the data

In [39]:
train = pd.read_csv('../Data/train.csv')
#ftest[ftest.columns[[0]+[i for i in range(10,18)]]]
#train.head()
def GetData(ds, splitper=0.2): #Splitter is stopped 
    np.random.seed(1337)
    #Convert The Percentage to split point
    splitper = 50 #int(math.floor(splitper * ds.shape[0] + 1))

    #Shuffle the list
    #Shuffle is stopped so we can get stable measurements
    #ds = ds.iloc[np.random.permutation(len(ds))]
    
    #Get tarin,test
    ls = [i for i in range(10,18)]
    ls+=[0,2]
    x_train = ds.iloc[splitper:][np.delete(ds.columns, ls)]
    y_train = ds.iloc[splitper:][ds.columns[10:18]]
    x_test = ds.iloc[:splitper][np.delete(ds.columns, ls)]
    y_test = ds.iloc[:splitper][ds.columns[10:18]]
    return (x_train.as_matrix(),y_train.as_matrix()),(x_test.as_matrix(),y_test.as_matrix()) 

def GetData2(ds, splitper=0.2): #Splitter is stopped 
    np.random.seed(1337)
    #Convert The Percentage to split point
    splitper = int(math.floor(splitper * ds.shape[0] + 1))
    scaler = MinMaxScaler(feature_range=(0, 1))
    ds = scaler.fit_transform(ds)
    #Shuffle the list
    #Shuffle is stopped so we can get stable measurements
    #ds = ds.iloc[np.random.permutation(len(ds))]
    
    #Get tarin,test
    ls = [i for i in range(10,18)]
    ls+=[0,2]
    
    x_train = ds[splitper:, np.delete(range(ds.shape[1]), ls)]
    y_train = ds[splitper:, 10:18]
    x_test = ds[:splitper, np.delete(range(ds.shape[1]), ls)]
    y_test = ds[:splitper, 10:18]
    
    return (x_train,y_train),(x_test,y_test)
def GetData3(ds):#, splitper=0.2): Splitter is stopped 
    np.random.seed(1337)
    #Convert The Percentage to split point
    splitper = 50 #int(math.floor(splitper * ds.shape[0] + 1))

    #Shuffle the list
    #Shuffle is stopped so we can get stable measurements
    #ds = ds.iloc[np.random.permutation(len(ds))]
    
    #Get tarin,test
    ls = [i for i in range(10,18)]
    ls+=[0,2]
    x_train = ds.iloc[splitper:][np.delete(ds.columns, ls)]*1000
    y_train = ds.iloc[splitper:][ds.columns[10:18]]*1000
    x_test = ds.iloc[:splitper][np.delete(ds.columns, ls)]*1000
    y_test = ds.iloc[:splitper][ds.columns[10:18]]*1000
    return (x_train.as_matrix(),y_train.as_matrix()),(x_test.as_matrix(),y_test.as_matrix()) 
def GetData4(ds):#, splitper=0.2): Splitter is stopped 
    np.random.seed(1337)
    #Convert The Percentage to split point
    splitper = 50 #int(math.floor(splitper * ds.shape[0] + 1))
    scaler = MinMaxScaler(feature_range=(0, 1000))
    ds = scaler.fit_transform(ds)
    #Shuffle the list
    #Shuffle is stopped so we can get stable measurements
    #ds = ds.iloc[np.random.permutation(len(ds))]
    
    #Get tarin,test
    ls = [i for i in range(10,18)]
    ls+=[0,2]
    
    x_train = ds[splitper:, np.delete(range(ds.shape[1]), ls)]
    y_train = ds[splitper:, 10:18]
    x_test = ds[:splitper, np.delete(range(ds.shape[1]), ls)]
    y_test = ds[:splitper, 10:18]
    
    return (x_train,y_train),(x_test,y_test)

In [40]:
train.head()

,GeneID,"absolute expression, parental strain t=0 (arbitrary units)",wt t=0,wt t=10,wt t=20,wt t=30,wt t=45,wt t=60,wt t=90,wt t=120,...,tfB-del t=90,tfB-del t=120,tfC-del t=0,tfC-del t=10,tfC-del t=20,tfC-del t=30,tfC-del t=45,tfC-del t=60,tfC-del t=90,tfC-del t=120
0,45,0.194,0.0,0.080,-0.021,-0.208,0.071,-0.066,-0.064,-0.062,...,0.082,-0.080,-0.185,-0.205,-0.072,-0.061,0.000,0.042,-0.147,-0.246
1,7767,1.480,0.0,-0.467,-0.376,-0.619,-0.199,-0.107,-0.143,-0.210,...,0.302,-0.148,-0.149,-0.383,0.183,-0.007,0.020,-0.052,0.074,0.188
2,3443,13.960,0.0,-0.313,-0.147,-0.641,-0.098,-0.200,-0.305,-0.083,...,-0.478,-0.611,-0.385,-0.591,-0.320,-0.110,-0.469,-0.443,-0.285,-0.356
3,5965,132.600,0.0,-0.183,-0.106,-0.448,-0.119,-0.167,-0.245,-0.194,...,-0.302,-0.473,-0.285,-0.493,-0.358,-0.405,-0.500,-0.344,-0.263,-0.376
4,1341,0.148,0.0,0.165,0.183,0.186,0.103,0.130,0.108,0.186,...,0.112,0.349,0.037,0.013,-0.006,0.057,0.053,0.163,-0.024,-0.065


In [42]:
print('Loading data...')
FUll_Results = pd.read_csv('Results.csv')
#(X_train, y_train), (X_test, y_test) =GetData()
(x_train,y_train),(x_test,y_test) = GetData2(train,0.1)
print('train:',x_train.shape,y_train.shape)
print('test: ',x_test.shape,y_test.shape)

Loading data...
train: (8356, 24) (8356, 8)
test:  (929, 24) (929, 8)


# Manual Testing
* Remember to select the data in the right shape for the model.

In [5]:
model = load_model('output_files/{}'.format(modelname))
print('Finished loading')
org = RANK(y_test)
pred = model.predict(x_test)
pred= RANK(pred)
predscore =Calculate_Score(pred,org)
orgscore = Calculate_Score(org,org)
print('C:{},P:{}'.format(predscore,orgscore))

NameError: name 'modelname' is not defined

# Test All Models Automatically on All the data
   * FUll_Results = pd.DataFrame(columns=['Model','Perfect Score','Predicted Score','Data Type'])

In [8]:
def Add_Info_DataSet(model,Predicted,Perfect,DataType):
    global FUll_Results
    FUll_Results = FUll_Results.append(pd.Series((model,Predicted,Perfect,DataType),index=['Model','Perfect Score','Predicted Score','Data Type']),ignore_index=True)

In [9]:
def StudyAModel(modelname):
    print('loading model')
    print(modelname)
    model = load_model('output_files/{}'.format(modelname))
    print('Finished loading')
    org = RANK(y_test)
    pred = model.predict(x_test)
    pred= RANK(pred)
    predscore =Calculate_Score(pred,org)
    orgscore = Calculate_Score(org,org)
    print('C:{},P:{}'.format(predscore,orgscore))
    #Add_Info_DataSet(modelname,predscore,orgscore,'TEST')
    #for i in range(0,184):
    #    x_tst = x_train[i*50:i*50+50]
    #    y_tst = y_train[i*50:i*50+50]
    #    org = RANK(y_tst)

    #    pred = model.predict(x_tst)
    #    pred = RANK(pred)
    #    predscore = Calculate_Score(pred,org)
    #    orgscore = Calculate_Score(org,org)
    #    if isinstance(predscore,str) or isinstance(orgscore,str):
    #        #problems.append('check{}, Score:{},{} '.format(i,str(predscore).strip(),str(orgscore).strip()))
    #        pass
    #    else:
    #        Add_Info_DataSet(modelname,predscore,orgscore,'TRN{}-{}'.format(i*50,i*50+50))
    print('Finished: {}'.format(modelname))

In [10]:
files=[]
for file in os.listdir("output_files/"):
    if file.endswith(".h5"):
        files.append(file)

In [11]:
#x_train = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
#x_test = np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))
#x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1], 1))
#x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1], 1))
for i in files:
    StudyAModel(i)

loading model
Model_LSTM12_03-1300.72.h5
Finished loading
C:[1] NA
,P:33.16822
Finished: Model_LSTM12_03-1300.72.h5
loading model
Model_LSTM12_02-1336.87.h5
Finished loading
C:[1] NA
,P:33.16822
Finished: Model_LSTM12_02-1336.87.h5
loading model
Model_LSTM12_00-1530.20.h5
Finished loading
C:[1] NA
,P:33.16822
Finished: Model_LSTM12_00-1530.20.h5
loading model
Model_LSTM12_04-1206.62.h5
Finished loading
C:[1] NA
,P:33.16822
Finished: Model_LSTM12_04-1206.62.h5


In [11]:
#FUll_Results.to_csv('Results.csv',index=False)

# Generate Best Results 

In [2]:
ftest = pd.read_csv('../Data/test.csv')

In [3]:
ls = [i for i in range(10,18)]
ls+=[0,2]
f_x_test = ftest.iloc[:][np.delete(ftest.columns, ls)].as_matrix()
f_x_test = np.reshape(f_x_test,(f_x_test.shape[0],1,f_x_test.shape[1]))
print('loading model')

model1 = load_model('output_files/Model_LSTM_07-0.06.h5')
model2 = load_model('output_files/Model_LSTM6_82-0.05.h5')
#model = load_model('output_files/Model_LSTM3_12-0.06.h5')
model3 = load_model('output_files/Model_LSTM3_28-0.06.h5')
print('Finished loading')

loading model
Finished loading


In [5]:
pred1= np.array(RANK(model1.predict(f_x_test)),dtype=int)
pred2= np.array(RANK(model2.predict(f_x_test)),dtype=int)
pred3= np.array(RANK(model3.predict(f_x_test)),dtype=int)

In [6]:
print(Calculate_Score(pred1,pred1))
print(Calculate_Score(pred2,pred2))
print(Calculate_Score(pred3,pred3))

32.90585
34.48005
32.38111


In [11]:
print(Calculate_Score(pred1,pred2))

21.22626


In [19]:
ftest.iloc[:,10:18]=pred1
newls = ls[:-2]
newls = [0]+newls
ftest[ftest.columns[newls]].to_csv('model1.txt',index=False,sep='\t')

In [20]:
ftest.iloc[:,10:18]=pred2
newls = ls[:-2]
newls = [0]+newls
ftest[ftest.columns[newls]].to_csv('model2.txt',index=False,sep='\t')

In [32]:
ftest.iloc[:,10:18]=pred3
newls = ls[:-2]
newls = [0]+newls
ftest[ftest.columns[newls]].to_csv('model3.txt',index=False,sep='\t')